In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import datetime
import matplotlib.pyplot as plt
from keras.utils import plot_model

In [ ]:
train_dir = '../input/fer2013/train/'
test_dir = '../input/fer2013/test/'

def get_count(path,set_):
    
    dict_ = {}
    
    for exp in os.listdir(path):
        
        dir_ = path + exp
        dict_[exp] = len(os.listdir(dir_))
        
    df = pd.DataFrame(dict_,index=[set_])
    
    return df
    

train_count = get_count(train_dir,'train')
test_count = get_count(test_dir,'test')

print(train_count)
print(test_count)

In [ ]:
train_count.transpose().plot(kind = 'bar')
test_count.transpose().plot(kind = 'bar')

In [ ]:
plt.figure(figsize=(14,22))
i = 1

for exp in os.listdir(train_dir):
    
    img = load_img((train_dir + exp + '/' + os.listdir(train_dir + exp)[1]))
    plt.subplot(1,7,i)
    plt.imshow(img)
#     print(img.size)
    plt.title(exp)
    plt.axis('off')
    i += 1
    
plt.show()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.3,horizontal_flip=True)

training_set = train_datagen.flow_from_directory(train_dir,
                                                 batch_size=64,
                                                 target_size=(48,48),
                                                 shuffle=True,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

testing_set = test_datagen.flow_from_directory(test_dir,
                                                 batch_size=64,
                                                 target_size=(48,48),
                                                 shuffle=True,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')

In [ ]:
# training_set.class_indices
testing_set.class_indices

In [ ]:
def get_model(input_size,classes=7):
    
    #initialising The CNN
    
    model = tf.keras.models.Sequential()
    
    model.add(Conv2D(32,
                     kernel_size=(3,3),
                     padding = 'same', 
                     activation = 'relu',
                     input_shape = input_size))
    
    model.add(Conv2D(64, 
                     kernel_size=(3,3),
                     padding='same',
                     activation='relu'))
    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, 
                     kernel_size=(3,3),
                     padding='same',
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)))
    
    model.add(Conv2D(256, 
                     kernel_size=(3,3),
                     activation='relu',
                     kernel_regularizer=regularizers.l2(0.01)))
    
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(512,
                     kernel_size=(3,3),
                     activation='relu',
                     padding='same',
                     kernel_regularizer=regularizers.l2(0.01)))
    model.add(Conv2D(512,
                     kernel_size=(3,3),
                     activation='relu',
                     padding='same',
                     kernel_regularizer=regularizers.l2(0.01)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(1024,activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(classes,activation='softmax'))
    
    model.compile(optimizer=Adam(lr=0.0001,decay=1e-6),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
fernet = get_model((48,48,1),7)
fernet.summary()

In [ ]:
plot_model(fernet,
           to_file='fernet.png',
           show_shapes=True,
           show_layer_names=True)

In [ ]:
#Callback functions

chk_path = 'ferNet.h5'
log_dir = 'checkpoint/logs/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

checkpoint = ModelCheckpoint(filepath=chk_path,
                             save_best_only=True,
                             verbose=1,
                             mode='min',
                             moniter='val_loss')

earlystop = EarlyStopping(monitor='val_loss', 
                          min_delta=0, 
                          patience=3, 
                          verbose=1, 
                          restore_best_weights=True)
                        
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2, 
                              patience=6, 
                              verbose=1, 
                              min_delta=0.0001)

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
csv_logger = CSVLogger('training_log')

callbacks = [checkpoint,reduce_lr,csv_logger]

In [ ]:
#training model

# print(training_set.n,training_set.batch_size)
steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = testing_set.n // testing_set.batch_size

hist = fernet.fit(x=training_set,
                 validation_data=testing_set,
                 epochs=100,
                 callbacks=callbacks,
                 steps_per_epoch=steps_per_epoch,
                 validation_steps=validation_steps)

In [ ]:
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
train_loss, train_accu = fernet.evaluate(training_set)
test_loss, test_accu = fernet.evaluate(testing_set)
print("final train accuracy = {:.2f} , validation accuracy = {:.2f}".format(train_accu*100, test_accu*100))

In [ ]:
fernet.save_weights('fernet_bestweight.h5')

In [ ]:
#Confusion Matrix and Classification on training set

y_pred = fernet.predict(training_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = testing_set.class_indices
class_labels = {v:k for k,v in class_labels.items()}

from sklearn.metrics import classification_report, confusion_matrix
cm_train = confusion_matrix(training_set.classes, y_pred)
print('Confusion Matrix')
print(cm_train)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(training_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_train, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)


In [ ]:
#Confusion Matrix and Classification on test set

y_pred = fernet.predict(testing_set)
y_pred = np.argmax(y_pred, axis=1)
class_labels = testing_set.class_indices
class_labels = {v:k for k,v in class_labels.items()}

#from sklearn.metrics import classification_report, confusion_matrix
cm_test = confusion_matrix(testing_set.classes, y_pred)
print('Confusion Matrix')
print(cm_test)
print('Classification Report')
target_names = list(class_labels.values())
print(classification_report(testing_set.classes, y_pred, target_names=target_names))

plt.figure(figsize=(8,8))
plt.imshow(cm_test, interpolation='nearest')
plt.colorbar()
tick_mark = np.arange(len(target_names))
_ = plt.xticks(tick_mark, target_names, rotation=90)
_ = plt.yticks(tick_mark, target_names)